In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import gzip
import os
from random import sample 
import subprocess
from pathlib import Path
import io

In [2]:
test0 = 'https://apkpure.com/root-car-insurance-good-drivers-save-money/com.joinroot.root'

In [3]:
base_url = 'https://apkpure.com'

In [4]:
xml_url = base_url + '/sitemap.xml'
r = requests.get(xml_url)

In [5]:
soup = BeautifulSoup(r.text, 'lxml')
xml_links_tag = soup.find_all('loc')
xml_links = []
for xml_link in xml_links_tag:
    xml_links.append(xml_link.text)

In [6]:
sampled_xml = sample(xml_links,5)
sampled_xml

['https://apkpure.com/sitemaps/education-520.xml.gz',
 'https://apkpure.com/sitemaps/art_and_design-98.xml.gz',
 'https://apkpure.com/sitemaps/shopping-131.xml.gz',
 'https://apkpure.com/sitemaps/education-555.xml.gz',
 'https://apkpure.com/sitemaps/house_and_home-18.xml.gz']

In [7]:
xml_file_paths = defaultdict(list)
for sample_xml in sampled_xml:
    #unzipping xml files 
    r = requests.get(sample_xml)
    compressed_gz = io.BytesIO(r.content)
    decompressed_gz = gzip.GzipFile(fileobj=compressed_gz)
    #getting file name 
    xml_name = sample_xml.split('/')[-1].replace('.gz', '')
    #getting category of app
    category = xml_name.split('-')[0]
    #create xml files folder if it does not exist
    if not os.path.exists('./xml_files'):
        os.mkdir('./xml_files')
    #create category folders if they dont exist
    if not os.path.exists('./xml_files/' + category):
        os.mkdir('./xml_files/' + category)
    #saving content of unzipped xml files 
    xml_out_path = './xml_files/' + category + '/' + xml_name
    xml_file_paths[category].append(xml_out_path)
    with open(xml_out_path, 'wb') as outfile:
        outfile.write(decompressed_gz.read())

In [8]:
app_links = defaultdict(list)
for category, xml_paths in xml_file_paths.items():
    for xml_path in xml_paths:
        with open(xml_path, "r") as f:
            contents = f.read()
            soup = BeautifulSoup(contents, 'lxml')
            app_link_loctag = soup.find_all('loc')
            for app_link_lt in app_link_loctag:
                app_links[category].append(app_link_lt.text)

In [9]:
sampled_apps = defaultdict(list)
for k,v in app_links.items():
    for l in (sample(v,5)):
        sampled_apps[k].append(l)

In [13]:
sampled_app_links

['https://apkpure.com/apprendre-l-allemand-facilement-et-rapidement/com.apgems.apprendreallemandrap',
 'https://apkpure.com/confucio-frases-inspiradoras/com.tonsu.confuciofrasesinspiradoras',
 'https://apkpure.com/ipseoa-federico-di-svevia/appinventor.ai_fabio_cianfagna.IPSEOA_ver2',
 'https://apkpure.com/%EC%8B%A4%EB%B2%84%EB%A7%8C%EC%84%B8/com.silvermanse.silvermanse',
 'https://apkpure.com/%D8%B3%D9%84%D8%B3%D9%84%D8%A9-%D8%A3%D8%AD%D9%83%D8%A7%D9%85-%D8%A7%D9%84%D8%AA%D8%AC%D9%88%D9%8A%D8%AF-%D8%A7%D9%84%D8%B4%D9%8A%D8%AE-%D8%A3%D8%AD%D9%85%D8%AF-%D8%B9%D8%A7%D9%85%D8%B1-mp3/com.voonoon.tajweed_ahmed_amer']

In [11]:
sampled_app_links = [test0]

In [12]:
for category, sampled_app_links in sampled_apps.items():

    #create category folders if they dont exist
    if not os.path.exists('./APK/' + category):
        os.mkdir('./APK/' + category)
    for sampled_app_link in sampled_app_links:

        url_split = sampled_app_link.split('/')
        if 'group' == url_split[3]:
            app_name = url_split[4]
        else:
            app_name = url_split[3]
        from urllib.request import Request, urlopen
        r = requests.get(sampled_app_link)
        soup = BeautifulSoup(r.text, 'html.parser')
        download_link = soup.find('div', class_ = 'ny-down')
        if download_link != None:
            download_url = base_url + (download_link.find('a', class_ = 'da')['href'])
        else:
            download_url = base_url + (soup.find('div', class_ = 'down').find('a')['href'])
        
        try:

            r = requests.get(download_url)
            soup = BeautifulSoup(r.text, 'html.parser')

            download_file = soup.find_all('a', class_ = "ga")[0]['href']
            r = requests.get(download_file, stream = True)
            data = r.content
            with open('./APK/' + category + '/' + app_name +'.apk', 'wb') as fh:
                fh.write(data)

        except:
            print(download_url)
            print('fail')


FileNotFoundError: [Errno 2] No such file or directory: './APK/education'

In [74]:
pathlist = Path('./APK').glob('**/*.apk')
for i,path in enumerate(pathlist):
    path_in_str = str(path)
    app_name = path_in_str.split('/')[2]
    directory = '/'.join(path_in_str.split('/')[:2])
    os.chdir(directory)
    test = subprocess.Popen(["/usr/local/bin/apktool","d", app_name ], stdout=subprocess.PIPE)
    output = test.communicate()[0]
    os.chdir('./../../')

In [79]:
# app_links = {}
# for cat_url in category_links:
#     cat = cat_url.split('/')[-1]
#     cat_url += '?sort=download'
#     app_links[cat] = []
#     r = requests.get(cat_url)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     for app_link in (soup.find_all('div', class_ = 'category-template-img')):
#         app_links[cat].append(base_url + app_link.find('a', href = True)['href'] + '/download?from=details')


In [22]:
import sys
sys.path.append('../')
import src.scrape_apk as s

import os

os.chdir('../')

In [23]:
s.run_scraper(['beauty'], 50,'./data')

beauty


FileNotFoundError: [Errno 2] No such file or directory: './data/xml_files'

In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
url = 'https://apkpure.com/moving-gifs-for-whatsapp-%F0%9F%8E%AC/com.GifsForWhatssap.GifMaker/download/2-APK?from=versions%2Fversion'

In [20]:
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [21]:
r.status_code

200